In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import math
from scipy.optimize import curve_fit, minimize
from itertools import combinations, product
from scipy.interpolate import griddata
from scipy.fftpack import fft2, ifft2, fftshift, ifftshift, fftfreq
from matplotlib.colors import Normalize
from scipy.misc import derivative
from multiprocessing import Pool
import pickle
import os
import copy

from IPython.display import clear_output
print(os.getcwd())

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        #print("Created Directory : ", directory)
    else:
        print("Directory already existed : ", directory)
    return directory

Quasi phase field solution for dislocation configuration.

<img src="illust.png" width="800px"/>

In [ ]:
def u_x(x, disso_x, wx, b):
    #* initial trial function for 1d displacement field
    return b/np.pi/2*(
        np.arctan((x-disso_x/2)/wx) + 
        np.arctan((x+disso_x/2)/wx)
    ) + b/2

def e_ext(tau, ux):
    return np.sum(tau*ux)

def ft_main(xy, *coef, n_coef=8, print_=False):
    #* x, y should be normalized by a / c
    x, y = xy[:,0]*2, xy[:,1]*2
    c = coef[0]
    n = (len(coef)-1)//n_coef
    
    y_ft = c
    prefac = 1/2
    for i in range(1, n+1):
        for j_ in range(1,n_coef//8+1):
            j = 8*j_
            y_ft += \
                coef[n_coef*i-j+1]*np.sin(prefac**j_*i*np.pi*x + prefac**j_*i*np.pi*y) \
              + coef[n_coef*i-j+2]*np.cos(prefac**j_*i*np.pi*x + prefac**j_*i*np.pi*y) \
              + coef[n_coef*i-j+3]*np.sin(prefac**j_*i*np.pi*x - prefac**j_*i*np.pi*y) \
              + coef[n_coef*i-j+4]*np.cos(prefac**j_*i*np.pi*x - prefac**j_*i*np.pi*y) \
              + coef[n_coef*i-j+5]*np.sin(prefac**j_*i*np.pi*x) \
              + coef[n_coef*i-j+6]*np.cos(prefac**j_*i*np.pi*x) \
              + coef[n_coef*i-j+7]*np.sin(prefac**j_*i*np.pi*y) \
              + coef[n_coef*i-j+8]*np.cos(prefac**j_*i*np.pi*y)
            
            if print_:
                print(prefac**j_*i)
            
    return y_ft

def e_total(uxy, #* combination of ux_13 and ux_23
            tau_13, tau_23,
            zero_k_ind, lattice_mesh,
            f13_mesh, f23_mesh,
            g13_mesh, g23_mesh,
            gsfe_coef, n_coef, #* gsfe part
            b_13, b_23, 
            mode = 'get_parts'
            ):

    def mirro(x, mirro_ind_x, mirro_ind_y):
        x_mirro = x.copy()
        x_mirro = np.concatenate((x, x[mirro_ind_x]), axis=0)
        x_mirro = np.concatenate((x_mirro, x_mirro[:,mirro_ind_y]), axis=1)
        return x_mirro
    
    lattice_dim = lattice_mesh.shape
    ny = lattice_dim[1]
    nx = lattice_dim[0]
    ux_13, ux_23 = uxy[:nx*ny], uxy[nx*ny:]
    
    #* ux_13 part
    #* boundary condition, otherwise fail to converge
    ux_13 = ux_13.reshape(lattice_dim[0], lattice_dim[1])
    ux_23 = ux_23.reshape(lattice_dim[0], lattice_dim[1])
    
    #* add boundary condition
    ux_13[0] = np.ones(ny)*0 
    ux_13[-1] = np.ones(ny)*b_13
    ux_23[0] = np.ones(nx)*0 
    ux_23[-1] = np.ones(nx)*b_23
    
    #* mirroring to ensure PBC
    mirro_ind_x = np.arange(nx)[::-1]
    mirro_ind_y = np.arange(ny)[::-1]
    
    #! mirroring at top or bottom?
    k_ux_13 = fft2(mirro(ux_13, mirro_ind_x, mirro_ind_y))
    k_ux_23 = fft2(mirro(ux_23, mirro_ind_x, mirro_ind_y))
    stress_k_13 = f13_mesh*k_ux_13 + g13_mesh*k_ux_23
    stress_k_23 = f23_mesh*k_ux_13 + g23_mesh*k_ux_23
    #* FT(sigma)(0, 0) = 0
    stress_k_13[zero_k_ind[0]][zero_k_ind[1]] = np.array(0).astype(complex)
    stress_k_23[zero_k_ind[0]][zero_k_ind[1]] = np.array(0).astype(complex)
    
    #* elastic energy
    #TODO use Parseval's identity (only if PBC is originally satisfied)
    # e_el_ = np.real(np.sum(stress_k*np.conj(k_ux))/lattice_dim[0]/lattice_dim[1]/2)
    
    #TODO direct in r-space
    stress_r_13 = ifft2(stress_k_13)
    e_el_13 = np.sum(np.real(stress_r_13[:nx, :ny])*ux_13)/2
    stress_r_23 = ifft2(stress_k_23)
    e_el_23 = np.sum(np.real(stress_r_23[:nx, :ny])*ux_23)/2
    
    ux_13_ravel = ux_13.flatten()
    ux_23_ravel = ux_23.flatten()
    ux_frac = np.array([
        (ux_13_ravel/b_13) % 1,
        (ux_23_ravel/b_23) % 1
    ]).T
    # print(np.max(ux_frac), np.min(ux_frac))
    ux_frac[ux_frac > 1/2] = 1 - ux_frac[ux_frac > 1/2]
    
    #* misfit
    gamma_xy = ft_main(ux_frac, *gsfe_coef, n_coef=n_coef, print_=False)*mjm2eva
    e_ms_ = np.sum(gamma_xy)
    
    #* external stress 
    e_tau_ = e_ext(tau_13, ux_13) + e_ext(tau_23, ux_23)
    
    #* penalty term for minus gradient
    # grad_bd = np.concatenate((np.gradient(ux[:bd_x], axis=0), np.gradient(ux[-bd_x:], axis=0)), axis=0)
    # grad_minus = np.sum(grad_bd[grad_bd < 0])
    #* penalty coef
    # e_alpha = 1e3
    # e_penalty = e_alpha*grad_minus
    e_penalty = 0
    
    #TODO displacement field gradient term 
    grad_epsilon = 0
    e_grad = grad_epsilon*np.sum(np.gradient(ux_13, axis=0)**2 + np.gradient(ux_23, axis=0)**2)
    
    if mode == 'get_parts':
        return [stress_k_13, stress_k_23], [stress_r_13, stress_r_23], \
            e_ms_, [e_el_13, e_el_23], e_tau_, e_grad
    else:
        return e_ms_ + e_el_13 + e_el_23 + e_tau_ - e_penalty + e_grad

def supercell_2d(frac, nx, ny):
    frac_raw, frac_n = frac.copy(), np.empty((0,2))
    nx_half, ny_half = nx//2, ny//2
    for i, j in product(range(-nx_half, nx_half+1), range(-ny_half, ny_half+1)):
        frac_n = np.concatenate((frac_n, frac_raw + np.array([i,j])))
    return frac_n
    
#* some constants
mjm2eva = 6.24150965*1e-5 #* mJ/m^2 to eV/Å^2
eva2gpa = 160.2176621 #* eV/Å^3 to GPa
specie_denote = '' #TODO specie denotion

#* some switches
run = True #TODO
load_runs_dict = False #TODO load previous runs_dict

#* lattice initialize
nx, ny = 0, 0 #TODO nx*ny grids
a, c = 0, 0 #TODO lattice constant
ab_angle = np.pi-2*np.arctan(c/a)
b = np.sqrt(a**2+c**2)

#* cij part 
cij = np.load(f'')
cij = cij/eva2gpa #* convert to eV/Å^3

#* gsfe part
n_coef = 8 #TODO specify the gsfe coefficients
gsfe_coef = np.load(f'') 

#* in real lattice the minimum spacing from 1100 projection will be b/4
fine_grid_n = 2 #TODO resolution of grid
lx, ly = a*nx/fine_grid_n, c*ny/fine_grid_n
prim_cell_frac = np.array([[0.,0.]])

#* orthogonal approximation: for fft grid
prim_cell_vec = np.array([
    [a/fine_grid_n, 0],
    [0, c/fine_grid_n],
])

len_nx, len_ny = nx, ny
lattice_grid_2d = np.zeros((len_nx, len_ny, 2))
half_nx, half_ny = len_nx//2, len_ny//2
for i, j in product(range(-half_nx, half_nx+1), range(-half_ny, half_ny+1)):
    #* linear superposition assumption for displacement field
    lattice_grid_2d[i+half_nx,j+half_ny] = np.array([i*a/fine_grid_n, j*c/fine_grid_n])

#* initialize frequency grid 
#* scipy solution
len_kx, len_ky = len_nx*2, len_ny*2 #* mirroring for PBC
kx, ky = fftfreq(len_kx, lx/len_nx)*2*np.pi, fftfreq(len_ky, ly/len_ny)*2*np.pi
kxy_mesh = np.array(list(product(kx, ky)))
#* find zero index (set self-stress as zero)
zero_kx = np.where(np.abs(kx) < 1e-10)[0][0]
zero_ky = np.where(np.abs(ky) < 1e-10)[0][0]
zero_k_ind = [zero_kx, zero_ky]

#* initialize displacment field in 2d form (for fft convenience), assume linear superposition
#* hint: a good initial guess can be obtained from a quick SVPN evaluation
d_init, w_init = 50, 1
ux_grid_2d = np.zeros((len_nx, len_ny))
uy_grid_2d = np.zeros((len_nx, len_ny))
for i, j in product(range(-half_nx, half_nx+1), range(-half_ny, half_ny+1)):
    ux_grid_2d[i+half_nx,j+half_ny] = u_x(i*a/fine_grid_n, d_init, w_init, a) #* x-axis u
    uy_grid_2d[i+half_nx,j+half_ny] = u_x(i*c/fine_grid_n, d_init, w_init, c) #* y-axis u

#* debug part
ux_trial = ux_grid_2d.copy()
ux_trial_raw = copy.deepcopy(ux_trial)
k_ux_trial = fft2(ux_trial)
ik_ux_trial = ifft2(k_ux_trial)

#* load prefactor for stress field
prefac_savpth = f''
kz_grid = ''
lattice_size = f'{nx}_{ny}'
f13_mesh = np.load(prefac_savpth+f'/{specie_denote}_{kz_grid}_f{fine_grid_n}_lat{lattice_size}_f13.npy')
f23_mesh = np.load(prefac_savpth+f'/{specie_denote}_{kz_grid}_f{fine_grid_n}_lat{lattice_size}_f23.npy')
g13_mesh = np.load(prefac_savpth+f'/{specie_denote}_{kz_grid}_f{fine_grid_n}_lat{lattice_size}_g13.npy')
g23_mesh = np.load(prefac_savpth+f'/{specie_denote}_{kz_grid}_f{fine_grid_n}_lat{lattice_size}_g23.npy')

#* main function 
def main(tau_13, tau_23):
    energy_info = []
    ux_info = []
    for d in []:
        
        #* initialize displacement field
        ux_grid_2d = np.zeros((len_nx, len_ny))
        uy_grid_2d = np.zeros((len_nx, len_ny))
        for i, j in product(range(-half_nx, half_nx+1), range(-half_ny, half_ny+1)):
            ux_grid_2d[i+half_nx,j+half_ny] = u_x(i*a/fine_grid_n, d, 1, a) #* x-axis u
            uy_grid_2d[i+half_nx,j+half_ny] = u_x(i*c/fine_grid_n, d, 1, c) #* y-axis u
            
        res = minimize(
            e_total,
            x0 = np.concatenate((ux_grid_2d.flatten(), uy_grid_2d.flatten())),
            args = (tau_13, tau_23, 
                    zero_k_ind, lattice_grid_2d,
                    f13_mesh, f23_mesh,
                    g13_mesh, g23_mesh,
                    gsfe_coef, n_coef, 
                    a, c, 
                    'None'),
            method='L-BFGS-B',
            bounds=bounds,
            tol=1e-8,
            options={"ftol":1e-6, "gtol":1e-6}
        )
        ux_info.append(res.x)
        energy_info.append(res.fun)
    
    energy_info = np.array(energy_info)
    e_tot = energy_info
    e_best = e_tot[np.argmin(e_tot)]
    ux_best = ux_info[np.argmin(e_tot)]
    
        
    return [tau_13, tau_23], e_best, ux_best

#* test seperate energy based on initial guess
tau_13_raw, tau_23_raw = 10/eva2gpa, 0
uxy_raw = np.concatenate((ux_grid_2d.flatten(), uy_grid_2d.flatten()))
stress_k_raw, stress_r_raw, e_ms_raw, e_el_raw, e_tau_raw, e_grad_raw = \
    e_total(uxy_raw,
            tau_13_raw, tau_23_raw, 
            zero_k_ind, lattice_grid_2d,
            f13_mesh, f23_mesh,
            g13_mesh, g23_mesh,
            gsfe_coef, n_coef, 
            a, c, 
            'get_parts')

best_ux_list = []
#* set bounds for optimization
bounds = tuple([[0,a] for _ in range(len(ux_grid_2d.flatten()))]) + \
            tuple([[0,c] for _ in range(len(uy_grid_2d.flatten()))])
np.random.seed(888)
print(f'number of grids: {ux_trial.shape}, number of k points: {kxy_mesh.shape}')
print(f'initial misfit E: {e_ms_raw}, elastic E: {e_el_raw}, external E: {e_tau_raw}, grad E: {e_grad_raw}')
print('Start to optimize')

if load_runs_dict:
    runs_dict_savpth = f'runs/{specie_denote}'
    with open(f'{runs_dict_savpth}/{specie_denote}_{nx}_{ny}.pickle', 'rb') as f:
        runs_dict = pickle.load(f)
else:
    runs_dict = {}
if run:
    tau_13_list = np.array([0, 5, 10, 15, 20, 25, 30, 35])/eva2gpa
    tau_23_list = np.zeros_like(tau_13_list)/eva2gpa
    tau_list = [(tau_13, tau_23) for tau_13, tau_23 in zip(tau_13_list, tau_23_list)]
    
    p = Pool(min(len(tau_list), 52))
    res = p.starmap(main, tau_list)
    tau_trial_list = [r[0] for r in res]
    best_ux_list = [r[2] for r in res]
    best_e_list  = [r[1] for r in res]
    
    for tau_, ux_best, e_best in zip(tau_trial_list, best_ux_list, best_e_list):
        tau = tau_
        tau_13, tau_23 = tau
        tau_denote = int(tau_13*eva2gpa)
    
        stress_k_best, stress_r_best, e_ms_best, e_el_best, e_tau_best, e_grad_best = \
            e_total(ux_best, 
                    tau_13, tau_23,
                    zero_k_ind, lattice_grid_2d,
                    f13_mesh, f23_mesh,
                    g13_mesh, g23_mesh,
                    gsfe_coef, n_coef, 
                    a, c, 
                    'get_parts')
        
        runs_dict[tau_denote] = {
            'e_best': e_best,
            'ux_best': ux_best,
            'stress_k_best': stress_k_best,
            'stress_r_best': stress_r_best,
            'e_ms_best': e_ms_best,
            'e_el_best': e_el_best,
            'e_tau_best': e_tau_best,
            'e_grad_best': e_grad_best,
        }
    
        print(f'tau: {tau_denote} GPa, E misfit: {e_ms_best}, E elastic: {e_el_best}, E external: {e_tau_best} E grad: {e_grad_best}')
        
runs_dict_savpth = f'runs/{specie_denote}'
create_dir(runs_dict_savpth)

with open(f'{runs_dict_savpth}/{specie_denote}_{nx}_{ny}.pickle', 'wb') as f:
    pickle.dump(runs_dict, f)